# West Coastal Plain & Ouachitas Open Pine Birds

This is an indicator for the 2023 base blueprint.

The WGCPO Open Pine Decision Support Model prioritizes management and protection actions within existing pine and mixed pine/hardwood habitat according to the landscape-scale needs of three Open Pine Umbrella Species (brown-headed nuthatch, Bachman’s sparrow, and red-cockaded woodpecker).

Created by Amy Keister, last run by Amy Keister on 28 February 2022. It took 55 minutes to run.

In [112]:
import os
import arcpy

In [113]:
import time
start = time.time()

In [114]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasOpen\West.gdb"

In [121]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasOpen\WestCoastalPlainAndOuachitasOpenPineBirds.tif"

In [116]:
# define sub-indicator outputs
BHNUout = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasOpen\BHNU_2019.tif"
BACSout = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasOpen\BACS_2019.tif"
RCWOout = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasOpen\RCWO_2019.tif"

In [117]:
# define rasters used for cell size, extent, and snapping
SEraster= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [118]:
# define inputs
FP= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\OpenPineManagementDST\OpenPineDSMInputs\floodplain_landform_combine.img"
NLCD= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
WGCPO= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\OpenPineManagementDST\OpenPineDSMInputs\WGCP_bcr_2010_Albers.shp"
# I'm using this just to help limit the extent of the current NLCD to save processing time and make sure it covers the
# whole buffer area
NLCDb= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\OpenPineManagementDST\OpenPineDSMInputs\2001_NLCD_NewWGCP_10kmbuff_USGSAlbers.img"

## Start Analysis

In [119]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [120]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasOpen\West.gdb


## Buffer study area to match previous run

In [10]:
# buffer study area
arcpy.analysis.Buffer(WGCPO, "WGCPO_Buffer", "10 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\West.gdb\\WGCPO_Buffer'>

In [11]:
# clip NLCD to buffered study area to match methods used in 2011
with arcpy.EnvManager(outputCoordinateSystem=NLCD, extent=NLCDb, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.ExtractByMask(NLCD, "WGCPO_Buffer"); out_raster.save("NLCD2")

## Remove floodplains from NLCD and calculate patches

In [12]:
# reclassify the floodplain layer
# I use environments here to set projection, snap,and cell size. I'm trying to be careful and not change the
# projection or snap on the NLCD source data at this point

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
with arcpy.EnvManager(outputCoordinateSystem=NLCD, extent=NLCDb, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.Reclassify(FP, "VALUE", "0 1;1 0", "DATA"); out_raster.save("FP2")

In [13]:
# pull out evergreen and mixed forest (42,43) from NLCD
# I use environments here to set projection, snap, and limit the analysis to the extent of the floodplain layer
out_raster = arcpy.sa.Con("NLCD2", 1, 0, "Value = 42 Or Value = 43"); out_raster.save("pine")

In [14]:
# remove all evergreen and mixed forest that occurs in floodplain
out_raster = arcpy.sa.Times("FP2", "pine"); out_raster.save("pineNoFP")

In [15]:
# reclassify to remove 0 values
out_raster = arcpy.sa.Reclassify("pineNoFP", "VALUE", "0 NODATA;1 1", "DATA"); out_raster.save("pineNoFP2")

In [16]:
# perform region group to idenify patches
out_raster = arcpy.sa.RegionGroup("pineNoFP2", "EIGHT", "WITHIN"); out_raster.save("pineNoFPrg")

In [17]:
# use lookup to pull out count values, so we can see the size of each patch in pixels
out_raster = arcpy.sa.Lookup("pineNoFPrg", "Count"); out_raster.save("pineNoFPrgCnt")

In [18]:
# take times 0.09 to convert from pixel count to hectares. 1 pixel = 30x30 meters = 900 sq meters = 0.09 ha
out_raster = arcpy.sa.Times("pineNoFPrgCnt", 0.09); out_raster.save("PatchHa")

## Perform brown-headed nuthatch analysis (BHNU)

In [19]:
# pull out patches greater than 3 hectares, which is the smallest patch that can still support 1 pair of BHNU
# this is different than my 2011 model, in that I make patches smaller than 3 ha NODATA instead of 0
out_raster = arcpy.sa.Con("PatchHa", "PatchHa", None, "VALUE >= 3"); out_raster.save("BHNUgt3_HaClump")

In [20]:
# Reclassify above raster so that I can do a euclidean distance on it, the result we need is all patches
# that are at least big enough to support a pair of BHNU to be 1 and everything else is NoData
out_raster = arcpy.sa.Reclassify("BHNUgt3_HaClump", "VALUE", "0 2000000 1", "DATA"); out_raster.save("BHNUgt3_NoData")

In [21]:
# calculate distance from BHNU patches, with a maximum distance of 460 meters, which is 1/2 the dispersal potential distance of 0.92 km
out_distance_raster = arcpy.sa.EucDistance("BHNUgt3_NoData", 460, 30, None, "PLANAR", None, None); out_distance_raster.save("BHNUgt3_NoData_EucDist460")

In [22]:
# recalssify euclidean distance to make everthing within 1500 m of a BACS patch 1 and everything else 0
out_raster = arcpy.sa.Con("BHNUgt3_NoData_EucDist460", 1, None, "VALUE >= 0"); out_raster.save("BHNUgt3_NoData_EucDist460_rcls")

In [23]:
# now we do a region group on these buffer areas around the BHNU patches. The reason for this is to find patches large
# enough to support at least 1 pair of BHNU that are within dispersal distance of each other
out_raster = arcpy.sa.RegionGroup("BHNUgt3_NoData_EucDist460_rcls", "FOUR", "WITHIN"); out_raster.save("BHNUgt3_NoData_EucDist460_rcls_clmp4")

In [24]:
# now we are using zonanal statistics to calculate the sum of pixels in patches that are large enough for at least
# 1 pair that are also within a dispersal distance of each other
# we go back to the BHNUgt3_NoData layer for the input because it has values of 1 for pixels in a patch big enough to 
# support at least # 1 pair and NoData for everything else. This is important so we can use the sum of those pixels 
# with a value of 1 to calculate area
out_raster = arcpy.sa.ZonalStatistics("BHNUgt3_NoData_EucDist460_rcls_clmp4", "Value", "BHNUgt3_NoData", "SUM", "DATA", "CURRENT_SLICE", 90); out_raster.save("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum")

In [25]:
# take times 0.09 to convert to hectares
out_raster = arcpy.sa.Times("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum", 0.09); out_raster.save("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_haPls")

In [26]:
# round down
out_raster = arcpy.sa.RoundDown("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_haPls"); out_raster.save("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_haPls2")

In [27]:
# convert to integer
out_raster = arcpy.sa.Int("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_haPls2"); out_raster.save("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_ha_int")

In [28]:
# if the area of pine habitat (in patches at least large enough to support 1 pair of BHNU) within a dispersal distance
# is greater than 84 (ha required to support a Minimum Viable Population) then make it 1, otherwise make it 0 
# this value is applying to the whole buffer area, not just the pine clumps
out_raster = arcpy.sa.Con("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_ha_int", 1, 0, "VALUE >= 84"); out_raster.save("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_ha_int_gt84")

In [29]:
# Now we want to only show the pine habitat in patches large enough to support at least one pair that are also in
# dispersal distances large enough to support a MVP. We do this with a times to reduce back to actual pine clumps large
# enough to support at least 1 pair
out_raster = arcpy.sa.Times("BHNUgt3_NoData_EucDist460_rcls_clmp4_ZstatSum_ha_int_gt84", "BHNUgt3_HaClump"); out_raster.save("BHNUgt3_HaClump_MVP")

In [30]:
# make all values greater than 84 have the value of 84, since these patches meet the MVP, we don't need to distinguish
# bigger patches that meet the MVP from smaller patches that meet the MVP
out_raster = arcpy.sa.Con("BHNUgt3_HaClump_MVP", 84, "BHNUgt3_HaClump_MVP", "VALUE >= 84"); out_raster.save("BHNUgt3_HaClump_MVP_84cut")

In [31]:
# reclassify the BHNU layer to combine with other layers
# 100 = patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support MVP
# 200 = patch is large enough to support 1 pair, and is within dispersal distance of enough habitat to support MVP
# 300 = patch alone is large enough to support MVP

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("BHNUgt3_HaClump_MVP", "VALUE", "NODATA 0;0 3 100;3 83 200;83 900000000 300", "DATA"); out_raster.save("BHNUgt3_HaClump_MVP_84cut_2clss")

In [32]:
# reclassify the BHNU layer for sub indicator output
# 1 = patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support MVP
# 2 = patch is large enough to support 1 pair, and is within dispersal distance of enough habitat to support MVP
# 3 = patch alone is large enough to support MVP

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("BHNUgt3_HaClump_MVP", "VALUE", "NODATA 0;0 3 1;3 83 2;83 900000000 3", "DATA"); out_raster.save("BHNUout1")

In [33]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = not an upland pine patch or an upland pine patch too small to support 1 pair'
    elif value == 1:
        return '1 = upland pine patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support a minimum viable population' 
    elif value == 2:
        return '2 = upland pine patch large enough to support 1 pair, and is within dispersal distance of enough habitat to support a minimum viable population'
    elif value == 3:
        return '3 = upland pine patch large enough to support a minimum viable population'
"""

In [34]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("BHNUout1", "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\West.gdb\\BHNUout1'>

In [35]:
# clip to WGCPO
with arcpy.EnvManager(outputCoordinateSystem=NLCD, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.ExtractByMask("BHNUout1", WGCPO); out_raster.save("BHNUout2")

In [36]:
# export as .tif 
arcpy.management.CopyRaster("BHNUout2", BHNUout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\BHNU_2019.tif'>

## Perform Bachman's sparrow analysis (BACS)

In [37]:
# pull out patches greater than 3 hectares, which is the smallest patch that can still support 1 pair of BACS
# this is different than my 2011 model, in that I make patches smaller than 3 ha NODATA instead of 0
out_raster = arcpy.sa.Con("PatchHa", "PatchHa", None, "VALUE >= 3"); out_raster.save("BACSgt3_HaClump")

In [38]:
# Reclassify above raster so that I can do a euclidean distance on it, the result we need is all patches
# that are at least big enough to support a pair of BACS to be 1 and everything else is NoData
out_raster = arcpy.sa.Reclassify("BACSgt3_HaClump", "VALUE", "0 2000000 1", "DATA"); out_raster.save("BACSgt3_NoData")

In [39]:
# calculate distance from BACS patches, with a maximum distance of 1500 meters, which is 1/2 the dispersal potential distance of 3 km
out_distance_raster = arcpy.sa.EucDistance("BACSgt3_NoData", 1500, 30, None, "PLANAR", None, None); out_distance_raster.save("BACSgt3_NoData_EucDist1500")

In [40]:
# recalssify euclidean distance to make everthing within 1500 m of a BACS patch 1 and everything else 0
out_raster = arcpy.sa.Con("BACSgt3_NoData_EucDist1500", 1, None, "VALUE >= 0"); out_raster.save("BACSgt3_NoData_EucDist1500_rcls")

In [41]:
# now we do a region group on these buffer areas around the BACS patches. The reason for this is to find patches large
# enough to support at least 1 pair of BACS that are within dispersal distance of each other
out_raster = arcpy.sa.RegionGroup("BACSgt3_NoData_EucDist1500_rcls", "FOUR", "WITHIN"); out_raster.save("BACSgt3_NoData_EucDist1500_rcls_clmp4")

In [42]:
# now we are using zonanal statistics to calculate the sum of pixels in patches that are large enough for at least
# 1 pair that are also within a dispersal distance of each other
# we go back to the BACSgt3_NoData layer for the input because it has values of 1 for pixels in a patch big enough to 
# support at least # 1 pair and NoData for everything else. This is important so we can use the sum of those pixels 
# with a value of 1 to calculate area
out_raster = arcpy.sa.ZonalStatistics("BACSgt3_NoData_EucDist1500_rcls_clmp4", "Value", "BACSgt3_NoData", "SUM", "DATA", "CURRENT_SLICE", 90); out_raster.save("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum")

In [43]:
# take times 0.09 to convert to hectares
out_raster = arcpy.sa.Times("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum", 0.09); out_raster.save("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_haPls")

In [44]:
# round down
out_raster = arcpy.sa.RoundDown("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_haPls"); out_raster.save("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_haPls2")

In [45]:
# convert to integer
out_raster = arcpy.sa.Int("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_haPls2"); out_raster.save("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_ha_int")

In [46]:
# if the area of pine habitat (in patches at least large enough to support 1 pair of BACS) within a dispersal distance
# is greater than 150 (ha required to support a Minimum Viable Population) then make it 1, otherwise make it 0 
# this value is applying to the whole buffer area, not just the pine clumps
out_raster = arcpy.sa.Con("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_ha_int", 1, 0, "VALUE >= 150"); out_raster.save("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_ha_int_gt150")

In [47]:
# Now we want to only show the pine habitat in patches large enough to support at least one pair that are also in
# dispersal distances large enough to support a MVP. We do this with a times to reduce back to actual pine clumps large
# enough to support at least 1 pair
out_raster = arcpy.sa.Times("BACSgt3_NoData_EucDist1500_rcls_clmp4_ZstatSum_ha_int_gt150", "BACSgt3_HaClump"); out_raster.save("BACSgt3_HaClump_MVP")

In [48]:
# make all values greater than 150 have the value of 150, since these patches meet the MVP, we don't need to distinguish
# bigger patches that meet the MVP from smaller patches that meet the MVP
out_raster = arcpy.sa.Con("BACSgt3_HaClump_MVP", 150, "BACSgt3_HaClump_MVP", "VALUE >= 150"); out_raster.save("BACSgt3_HaClump_MVP_150cut")

In [49]:
# reclassify the BACS layer to combine with other layers
# 10 = patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support MVP
# 20 = patch is large enough to support 1 pair, and is within dispersal distance of enough habitat to support MVP
# 30 = patch alone is large enough to support MVP

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("BACSgt3_HaClump_MVP", "VALUE", "NODATA 0;0 3 10;3 149 20;149 900000000 30", "DATA"); out_raster.save("BACSgt3_HaClump_MVP_150cut_2clss")

In [50]:
# reclassify the BACS layer for sub indicator output
# 1 = pine patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support a minimum viable population
# 2 = pine patch large enough to support 1 pair, and is within dispersal distance of enough habitat to support a minimum viable population
# 3 = pine patch large enough to support a minimum viable population

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("BACSgt3_HaClump_MVP_150cut", "VALUE", "NODATA 0;0 3 1;3 149 2;149 900000000 3", "DATA"); out_raster.save("BACSout1")

In [51]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = not an upland pine patch or an upland pine patch too small to support 1 pair'
    elif value == 1:
        return '1 = upland pine patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support a minimum viable population' 
    elif value == 2:
        return '2 = upland pine patch large enough to support 1 pair, and is within dispersal distance of enough habitat to support a minimum viable population'
    elif value == 3:
        return '3 = upland pine patch large enough to support a minimum viable population'
"""

In [52]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("BACSout1", "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\West.gdb\\BACSout1'>

In [53]:
# clip to WGCPO
with arcpy.EnvManager(outputCoordinateSystem=NLCD, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.ExtractByMask("BACSout1", WGCPO); out_raster.save("BACSout2")

In [54]:
# export as .tif 
arcpy.management.CopyRaster("BACSout2", BACSout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\BACS_2019.tif'>

## Perform red-cockaded woodpecker analysis (RCWO)

In [55]:
# pull out patches greater than 50 hectares, which is the smallest patch that can still support 1 pair of RCWO
# this is different than my 2011 model, in that I make patches smaller than 50 ha NODATA instead of 0
out_raster = arcpy.sa.Con("PatchHa", "PatchHa", None, "VALUE >= 50"); out_raster.save("RCWOgt50_HaClump")

In [56]:
# Reclassify above raster so that I can do a euclidean distance on it, the result we need is all patches
# that are at least big enough to support a pair of RCWO to be 1 and everything else is NoData
out_raster = arcpy.sa.Reclassify("RCWOgt50_HaClump", "VALUE", "0 2000000 1", "DATA"); out_raster.save("RCWOgt50_NoData")

In [57]:
# calculate distance from RCWO patches, with a maximum distance of 4000 meters, which is 1/2 dispersal potential distance if 8 km
out_distance_raster = arcpy.sa.EucDistance("RCWOgt50_NoData", 4000, 30, None, "PLANAR", None, None); out_distance_raster.save("RCWOgt50_NoData_EucDist4000")

In [58]:
# recalssify euclidean distance to make everthing within 4000 m of a RCWO patch 1 and everything else 0
out_raster = arcpy.sa.Con("RCWOgt50_NoData_EucDist4000", 1, None, "VALUE >= 0"); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls")

In [59]:
# now we do a region group on these buffer areas around the RCWO patches. The reason for this is to find patches large
# enough to support at least 1 pair of RCWO that are within dispersal distance of each other
out_raster = arcpy.sa.RegionGroup("RCWOgt50_NoData_EucDist4000_rcls", "FOUR", "WITHIN"); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls_clmp4")

In [60]:
# now we are using zonanal statistics to calculate the sum of pixels in patches that are large enough for at least
# 1 pair that are also within a dispersal distance of each other
# we go back to the RCWOgt50_NoData layer for the input because it has values of 1 for pixels in a patch big enough to 
# support at least # 1 pair and NoData for everything else. This is important so we can use the sum of those pixels 
# with a value of 1 to calculate area
out_raster = arcpy.sa.ZonalStatistics("RCWOgt50_NoData_EucDist4000_rcls_clmp4", "Value", "RCWOgt50_NoData", "SUM", "DATA", "CURRENT_SLICE", 90); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum")

In [61]:
# take times 0.09 to convert to hectares
out_raster = arcpy.sa.Times("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum", 0.09); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_haPls")

In [62]:
# round down
out_raster = arcpy.sa.RoundDown("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_haPls"); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_haPls2")

In [63]:
# convert to integer
out_raster = arcpy.sa.Int("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_haPls2"); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_ha_int")

In [64]:
# if the area of pine habitat (in patches at least large enough to support 1 pair of RCWO) within a dispersal distance
# is greater than 1000 (ha required to support a Minimum Viable Population) then make it 1, otherwise make it 0 
# this value is applying to the whole buffer area, not just the pine clumps
out_raster = arcpy.sa.Con("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_ha_int", 1, 0, "VALUE >= 1000"); out_raster.save("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_ha_int_gt1000")

In [65]:
# Now we want to only show the pine habitat in patches large enough to support at least one pair that are also in
# dispersal distances large enough to support a MVP. We do this with a times to reduce back to actual pine clumps large
# enough to support at least 1 pair
out_raster = arcpy.sa.Times("RCWOgt50_NoData_EucDist4000_rcls_clmp4_ZstatSum_ha_int_gt1000", "RCWOgt50_HaClump"); out_raster.save("RCWOgt50_HaClump_MVP")

In [66]:
# make all values greater than 1000 have the value of 1000, since these patches meet the MVP, we don't need to distinguish
# bigger patches that meet the MVP from smaller patches that meet the MVP
out_raster = arcpy.sa.Con("RCWOgt50_HaClump_MVP", 1000, "RCWOgt50_HaClump_MVP", "VALUE >= 1000"); out_raster.save("RCWOgt50_HaClump_MVP_1000cut")

In [67]:
# reclassify the RCWO layer to combine with other layers and for sub indicator output
# 1 = patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support MVP
# 2 = patch is large enough to support 1 pair, and is within dispersal distance of enough habitat to support MVP
# 3 = patch alone is large enough to support MVP

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("RCWOgt50_HaClump_MVP", "VALUE", "NODATA 0;0 50 1;50 999 2;999 900000000 3", "DATA"); out_raster.save("RCWOgt50_HaClump_MVP_1000cut_2clss")

In [68]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = not an upland pine patch or an upland pine patch too small to support 1 pair'
    elif value == 1:
        return '1 = upland pine patch large enough to support 1 pair, but outside of dispersal distance of enough habitat to support a minimum viable population' 
    elif value == 2:
        return '2 = upland pine patch large enough to support 1 pair, and is within dispersal distance of enough habitat to support a minimum viable population'
    elif value == 3:
        return '3 = upland pine patch large enough to support a minimum viable population'
"""

In [69]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("RCWOgt50_HaClump_MVP_1000cut_2clss", "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\West.gdb\\RCWOgt50_HaClump_MVP_1000cut_2clss'>

In [70]:
# clip to WGCPO
with arcpy.EnvManager(outputCoordinateSystem=NLCD, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.ExtractByMask("RCWOgt50_HaClump_MVP_1000cut_2clss", WGCPO); out_raster.save("RCWOout2")

In [71]:
# export as .tif 
arcpy.management.CopyRaster("RCWOout2", RCWOout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\RCWO_2019.tif'>

## New method to combine 3 species layers

In [72]:
# use cell statistics and sum to combine
out_raster = arcpy.sa.CellStatistics("BHNUgt3_HaClump_MVP_84cut_2clss;BACSgt3_HaClump_MVP_150cut_2clss;RCWOgt50_HaClump_MVP_1000cut_2clss", "SUM", "DATA", "SINGLE_BAND"); out_raster.save("SUM")

In [73]:
# clip to WGCPO
with arcpy.EnvManager(outputCoordinateSystem=NLCD, snapRaster=NLCD, cellSize=NLCD):
    out_raster = arcpy.sa.ExtractByMask("SUM", WGCPO); out_raster.save("SUM_WGCPO")

## Reclassify to the 6 class legend that the JV prefered

In [74]:
# reclassify the raster with the 3 combined species where values represent BHNU in the hundreds place, BACS in the tens, and RCWO in the ones to the preferred LMVJV legend option
out_raster = arcpy.sa.Reclassify("SUM_WGCPO", "Value", "0 0;110 0;120 1;121 1;122 2;210 1;220 2;221 2;222 3;312 4;321 4;322 4;331 5;332 5;333 6", "DATA"); out_raster.save(r"indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [133]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("indicator", SEraster); out_raster.save("SEMask2")

In [134]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("SEMask2", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [135]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Pine patch too small and isolated to support a population of any umbrella bird species or not an upland pine patch'
    elif value == 1:
        return '1 = Pine patch part of a cluster of nearby patches able to support a population of 1 umbrella bird species if managed in open condition'
    elif value == 2:
        return '2 = Pine patch part of a cluster of nearby patches able to support a population of 2 umbrella bird species if managed in open condition'
    elif value == 3:
        return '3 = Pine patch part of a cluster of nearby patches able to support a population of all 3 umbrella bird species if managed in open condition' 
    elif value == 4:
        return '4 = Pine patch large enough to support a population of 1 umbrella bird species if managed in open condition'
    elif value == 5:
        return '5 = Pine patch large enough to support a population of 2 umbrella bird species if managed in open condition'
    elif value == 6:
        return '6 = Pine patch large enough to support a population of all 3 umbrella bird species (brown-headed nuthatch, Bachman’s sparrow, red-cockaded woodpecker) if managed in open condition'
"""

In [136]:
# add and calculate description field to hold second option of indicator values
arcpy.management.CalculateField(Out, "Descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")


<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\WestCoastalPlainAndOuachitasOpenPineBirds.tif'>

In [137]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 6:
		return 73
	if Value == 5:
		return 139
	if Value == 4:
		return 205
	if Value == 3:
		return 247
	if Value == 2:
		return 251
	if Value == 1:
		return 253
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 6:
		return 0
	if Value == 5:
		return 1
	if Value == 4:
		return 35
	if Value == 3:
		return 104
	if Value == 2:
		return 172
	if Value == 1:
		return 215
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 6:
		return 106
	if Value == 5:
		return 121
	if Value == 4:
		return 143
	if Value == 3:
		return 161
	if Value == 2:
		return 185
	if Value == 1:
		return 211
	if Value == 0:
		return 255
	else:
		return 255
"""

In [138]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasOpen\\WestCoastalPlainAndOuachitasOpenPineBirds.tif'>

In [139]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

584.5379040241241
